In [ ]:
import numpy as np
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

def show_tensor_images(image_tensor, num_images=10, size=(1, 126, 126)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cp -av /content/drive/MyDrive/'Giraffa Analytics'/'MRI-Blur-Detection'/'Data' 'preprocessed slices'

'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data' -> 'preprocessed slices'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices' -> 'preprocessed slices/preprocessed slices'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC247_nomotion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC247_nomotion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC248_motion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC248_motion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC248_nomotion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC248_nomotion.npy'
'/content/drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/slice_14_NC249_motion.npy' -> 'preprocessed slices/preprocessed slices/slice_14_NC249_motion.npy'
'/content/drive/MyDrive/Giraffa 

In [ ]:
device='cuda'

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
import numpy as np
from PIL import Image

def convert_image_to_shape(image_path, target_shape):
    # Open the image from the specified path
    img = Image.fromarray(np.load(image_path))

    # Convert the PIL image to grayscale if the target shape specifies a single channel
    if img.mode != 'L' and target_shape[0] == 1:
        img = img.convert('L')

    # Resize the image to the target shape (width, height)
    img_resized = np.array(img.resize((target_shape[2], target_shape[1]), Image.LANCZOS))

    return img_resized.reshape((1, target_shape[2], target_shape[1]))/255

In [ ]:
img_shape = (1, 126, 126)

In [ ]:
from PIL import Image
import os

images = []
for img_path in os.listdir('drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices'):
    #print(img_path.split('_'))
    # Load the image using Pillow
    if  ("nomotion" in img_path) and int(img_path.split('_')[1]) >= 110 and int(img_path.split('_')[1]) <=130:
      image = convert_image_to_shape(f'drive/MyDrive/Giraffa Analytics/MRI-Blur-Detection/Data/preprocessed slices/{img_path}', target_shape=img_shape)
      images.append(torch.tensor(image).float())



In [ ]:
len(images)

609

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class VAE(nn.Module):
    def __init__(self, image_size=126, latent_dim=20):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * (image_size // 8) * (image_size // 8), latent_dim * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128 * (image_size // 8) * (image_size // 8)),
            nn.ReLU(),
            nn.Unflatten(1, (128, image_size // 8, image_size // 8)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encode
        x = self.encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=1)

        # Reparameterize
        z = self.reparameterize(mu, logvar)

        # Decode
        x_recon = self.decoder(z)

        return x_recon, mu, logvar




In [ ]:
import torch

def generate_samples(vae, n, latent_dim=20, device='cuda'):
    """
    Generate n data points using the specified VAE.

    Parameters:
    - vae: VAE model
    - n: Number of samples to generate
    - latent_dim: Dimension of the latent space
    - device: Device to run the generation on ('cuda' or 'cpu')

    Returns:
    - generated_samples: Tensor containing the generated samples
    """
    with torch.inference_mode():

        # Example usage
        input_image = torch.randn(n, 1, 126, 126).to(device)  # Batch size of n, single channel (grayscale), 126x126 image
        generated_samples, _, _ = vae(input_image)

    return generated_samples

In [ ]:
import torch.optim as optim

dataloader = DataLoader(CustomDataset(images), batch_size=32, shuffle=True)

# Instantiate the VAE
latent_dim = 20  # Adjust the latent dimension according to your requirements
vae = VAE(latent_dim=latent_dim).to(device)

# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='sum')  # Binary Cross Entropy loss for binary images
optimizer = optim.Adam(vae.parameters(), lr=0.001)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
# Training loop
num_epochs = 100
losses = []
for epoch in tqdm(range(num_epochs)):
    total_loss = 0.0
    for batch in dataloader:
        optimizer.zero_grad()

        # Forward pass
        x = batch.to(device)
        x_recon, mu, logvar = vae(x)

        x_recon_resized = nn.functional.interpolate(x_recon, size=(126, 126), mode='bilinear', align_corners=False)

        # Compute the loss
        BCE_loss = criterion(x_recon_resized, x.view(-1, 1, 126, 126))
        KLD_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        loss = BCE_loss + KLD_loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    ## Print the average loss for the epoch
    #print(f"Epoch {epoch + 1}, Average Loss: {total_loss / len(dataloader.dataset)}")

    losses.append(total_loss/len(dataloader.dataset))

    # Show real and fake images during training
    if epoch % 10 == 0:
      with torch.inference_mode():
          fake_images = generate_samples(vae, 10, latent_dim=latent_dim)
          real_images = next(iter(dataloader))[:10].to(device)
          show_tensor_images(fake_images)
          show_tensor_images(real_images)

    # Plot the losses
plt.plot(losses)
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

# Save the trained VAE
torch.save(vae.state_dict(), "./drive/MyDrive/GAN Stuff/vae.pth")